In [29]:
pip install pyarrow fastparquet s3fs

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import requests
import pandas as pd
from datetime import datetime
import pytz 

# API endpoint and parameters
WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "8b9f21b3769460637f69c4fedcb8915f"  # Replace with your actual API key


provinces = {
    "Chiang Mai":{
        "lat": 18.79038,
        "lon": 98.98468
    },
    "Lamphun":{
        "lat": 18.5745,
        "lon": 99.0087
    },
    "Lampang":{
        "lat": 18.29232,
        "lon": 99.49277
    },
    "Uttaradit":{
        "lat": 17.6255,
        "lon": 100.0942
    },
    "Phrae":{
        "lat": 18.1459,
        "lon": 100.1410
    },
    "Nan":{
        "lat": 18.793543,
        "lon": 100.743077
    },
    "Phayao":{
        "lat": 19.1920,
        "lon": 99.8788
    },
    "Chiang Rai":{
        "lat": 19.9086 ,
        "lon": 99.8325
    },
    "Mae Hong Son":{
        "lat": 19.3006 ,
        "lon": 97.9686
}
}
# Function to fetch and process weather data
def get_weather_data(province='Pathum Thani'):
    
    params = {
        "lat": provinces[province]['lat'],
        "lon": provinces[province]['lon'],
        "appid": API_KEY,
        "units": "metric"
    }
    try:
        # Make API request
        response = requests.get(WEATHER_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        
        # Convert timestamp to datetime
        # created_at = datetime.fromtimestamp(data['dt'])

        dt = datetime.now()
        thai_tz = pytz.timezone('Asia/Bangkok')
        created_at = dt.replace(tzinfo=thai_tz)

        # Convert timestamp to datetime
        timestamp = datetime.fromtimestamp(data['dt'])
        
        # Create dictionary with required fields
        weather_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'created_at': created_at,
            'requested_province':province,
            'location': data['name'],
            'weather_main': data['weather'][0]['main'],
            'weather_description': data['weather'][0]['description'],
            'main.temp': data['main']['temp'],
            'main.temp_min': data['main']['temp_min'],
            'main.temp_max': data['main']['temp_max'],
            'main.feels_like': data['main']['feels_like'],
            'main.pressure': data['main']['pressure'],
            'main.humidity': data['main']['humidity'],
            'main.sea_level': data['main']['sea_level'],
            'main.grnd_level': data['main']['grnd_level'],
            'visibility': data['visibility'],
            'wind.speed': data['wind']['speed'],
            'wind.deg': data['wind']['deg']
            
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return weather_dict
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None

In [37]:
df=pd.DataFrame([get_weather_data(p) for p in list(provinces.keys())])
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype                       
---  ------               --------------  -----                       
 0   timestamp            9 non-null      datetime64[ns]              
 1   year                 9 non-null      int64                       
 2   month                9 non-null      int64                       
 3   day                  9 non-null      int64                       
 4   hour                 9 non-null      int64                       
 5   minute               9 non-null      int64                       
 6   created_at           9 non-null      datetime64[ns, Asia/Bangkok]
 7   requested_province   9 non-null      object                      
 8   location             9 non-null      object                      
 9   weather_main         9 non-null      object                      
 10  weather_description  9 non-null      objec

,timestamp,year,month,day,hour,minute,created_at,requested_province,location,weather_main,...,main.temp_min,main.temp_max,main.feels_like,main.pressure,main.humidity,main.sea_level,main.grnd_level,visibility,wind.speed,wind.deg
0,2025-04-29 16:05:01,2025,4,29,16,5,2025-04-29 16:25:29.985446+07:00,Chiang Mai,Chiang Mai,Clouds,...,30.68,31.94,34.38,1007,51,1007,962,10000,3.09,190
1,2025-04-29 16:07:32,2025,4,29,16,7,2025-04-29 16:25:34.771275+07:00,Lamphun,Lamphun,Clouds,...,32.10,32.10,39.10,1007,66,1007,970,10000,2.17,245
2,2025-04-29 16:06:21,2025,4,29,16,6,2025-04-29 16:25:34.866185+07:00,Lampang,Lampang,Clouds,...,32.04,32.04,35.72,1009,55,1009,974,10000,1.54,0
3,2025-04-29 16:07:32,2025,4,29,16,7,2025-04-29 16:25:34.967679+07:00,Uttaradit,Uttaradit,Clouds,...,29.03,29.03,30.08,1008,53,1008,990,10000,2.84,223
4,2025-04-29 16:07:32,2025,4,29,16,7,2025-04-29 16:25:35.070775+07:00,Phrae,Phrae,Clouds,...,30.03,30.03,30.64,1009,47,1009,972,10000,2.28,270
5,2025-04-29 16:07:32,2025,4,29,16,7,2025-04-29 16:25:35.176630+07:00,Nan,Nan,Clouds,...,30.19,30.19,32.40,1009,56,1009,970,10000,1.81,276
6,2025-04-29 16:00:38,2025,4,29,16,0,2025-04-29 16:25:35.284914+07:00,Phayao,Phayao,Clouds,...,25.79,25.79,25.89,1008,56,1008,948,10000,2.30,222
7,2025-04-29 16:04:32,2025,4,29,16,4,2025-04-29 16:25:35.384159+07:00,Chiang Rai,Chiang Rai,Clouds,...,28.92,28.92,30.61,1010,58,1010,952,8000,2.06,120
8,2025-04-29 16:07:33,2025,4,29,16,7,2025-04-29 16:25:35.492456+07:00,Mae Hong Son,Mae Hong Son,Clouds,...,28.97,28.97,28.96,1006,44,1006,939,10000,2.17,165


In [39]:
import pandas as pd

# lakeFS credentials from your docker-compose.yml
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"

# lakeFS endpoint (running locally)
# lakefs_endpoint = "http://lakefs-dev:8000/"
lakefs_endpoint = "http://localhost:8001/"

# lakeFS repository, branch, and file path
repo = "weather"
branch = "main"
path = "weather.parquet"

# Construct the full lakeFS S3-compatible path
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

# Configure storage_options for lakeFS (S3-compatible)
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}

In [40]:
df.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    partition_cols=['year','month','day','hour']
)